# Deep Learning - Simple

---

# 1 MNIST 손글씨 인식

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.rc('figure', figsize=(10, 6))

from matplotlib import rcParams
rcParams['font.family'] = 'New Gulim'
rcParams['font.size'] = 10
rcParams['axes.unicode_minus'] = False

#### 패키지 임포트

In [ ]:
from keras.utils import np_utils

from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint,EarlyStopping

import tensorflow as tf

In [ ]:
# seed 값 설정
np.random.seed(0)
tf.random.set_seed(3)

#### 데이터 로드 및 분할

In [ ]:
# MNIST 데이터 불러오기
from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 784).astype('float32') / 255
X_test = X_test.reshape(X_test.shape[0], 784).astype('float32') / 255

y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

#### 모델 생성 및 설정

In [ ]:
model = Sequential()
model.add(Dense(512, input_dim=784, activation='relu'))
model.add(Dense(10, activation='softmax'))

#### 모델 계층 확인

In [ ]:
model.summary()

#### 모델 컴파일

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#### 모델 업데이트 및 저장 설정

In [ ]:
modelpath='./model/model_mnist_simple_{epoch:02d}-{val_loss:.4f}.hdf5'
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)

#### 학습 자동 중단 설정

In [ ]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

#### 모델 학습 실행 및 저장

In [ ]:
%%time
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30, batch_size=200, verbose=0,
                    callbacks=[early_stopping_callback,checkpointer])

#### 모델 평가

In [ ]:
# 테스트 정확도 출력
print('\n Test Accuracy: %.4f' % (model.evaluate(X_test, y_test)[1]))

#### 학습 진행 과정
- history
 - loss: 훈련 손실값
 - accuracy: 훈련 정확도
 - val_loss: 검증 손실값
 - val_accuracy: 검증 정확도

In [ ]:
# 그래프로 표현
plt.plot(history.history['val_loss'], marker='.', c='red', label='Validation Loss')
plt.plot(history.history['loss'],     marker='.', c='blue',label='Train Loss')

plt.grid()
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')

plt.title('학습 진행에 따른 학습 데이터와 검증 데이터의 에러')
plt.show()

#### 결과 예측

In [ ]:
# 예측 확률
pred_prob = model.predict(X_test)
pred_prob

In [ ]:
# 결과 예측
pred = np.argmax(pred_prob, axis=1).flatten()
pred

#### 결과 확인

In [ ]:
plt.imshow(X_test[0].reshape(28,-1))
plt.title(f'Label: {pred[0]}')
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(10, 5), subplot_kw={'xticks':(), 'yticks': ()})
for i, (ax, img) in enumerate(zip(axes.ravel(), X_test[:10].reshape(10, 28,-1))):
    ax.imshow(img)
    ax.set_xlabel(f'Label: {pred[i]}')

#### 결과 평가

In [ ]:
y_test_1d = np.argmax(y_test, axis=1).flatten()

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test_1d, pred))

---

In [ ]:
# End of file